In [1]:
# 파일 갯수 확인
import os
dataset_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/data/current_dataset'

# 쓰레기 카테고리
categories = ['glass', 'metal', 'paper', 'vinyl']
subcategories = {'plastic': ['label', 'no_label']}

def count_images_in_folder(folder_path):
    """ 해당 폴더 내의 이미지 파일 수를 세는 함수 """
    return len([file for file in os.listdir(folder_path) if file.lower().endswith(('.png', '.jpg', '.jpeg'))])

# 메인 카테고리별 이미지 수 계산
for category in categories:
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        count = count_images_in_folder(category_path)
        print(f"{category}: {count} images")

# 플라스틱 서브카테고리별 이미지 수 계산
for subcategory in subcategories['plastic']:
    subcategory_path = os.path.join(dataset_path, 'plastic', subcategory)
    if os.path.isdir(subcategory_path):
        count = count_images_in_folder(subcategory_path)
        print(f"plastic/{subcategory}: {count} images")

glass: 1420 images
metal: 1614 images
paper: 1640 images
vinyl: 1362 images
plastic/label: 612 images
plastic/no_label: 543 images


In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.6 MB/s eta 0:00:00


In [3]:


import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import os
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
import keras_tuner
from keras_tuner import RandomSearch
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from kerastuner import HyperParameters as hp
import glob
from tensorflow.keras.optimizers import Adam, SGD, RMSprop



# 데이터셋 경로
dataset_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/data/current_dataset'

# 이미지 크기 및 배치 크기 설정
img_width, img_height = 224, 224  # ResNet50의 기본 이미지 크기


# 모델 생성
def build_model(hp):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=hp.Int('units', min_value=512, max_value=1024, step=32),
              activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid']))(x)
    x = Dropout(rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1))(x)
    predictions = Dense(5, activation='softmax')(x)

    # 학습률 및 옵티마이저 튜닝
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# 배치사이즈 / ImageDataGenerator / train,val 분류
def create_generators(hp):
    batch_size = hp.Int('batch_size', min_value=32, max_value=128, step=32)

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, validation_generator

# 하이퍼파라미터 및 튜너 설정
hyperparameters = keras_tuner.HyperParameters()

# 데이터 생성기 생성
train_generator, validation_generator = create_generators(hyperparameters)


# 튜너 설정
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=15,  # 시도할 최대 트라이얼 수
    executions_per_trial=3,  # 각 트라이얼마다의 실행 횟수
    # 튜닝 결과 저장 경로
    directory='/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/hpt',
    # 폴더 이름
    project_name='ResNet50_hpt_2',
    hyperparameters=hyperparameters,
    overwrite=False  # 이전 튜닝 세션을 재개하기 위해 False로 설정
)

# 튜너의 결과를 로드합니다.
tuner.reload()

# 최적의 하이퍼파라미터를 가져옵니다.
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print('\nBest Hyperparameters:')
print(best_hps.values,end='\n\n')



for trial in tuner.oracle.trials.values():
    # 메트릭 중 'val_accuracy'가 존재하는지 확인
    if 'val_accuracy' in trial.metrics.metrics:
        val_accuracy = trial.metrics.get_best_value('val_accuracy')
        print(f'Trial ID: {trial.trial_id}, Best Validation Accuracy: {val_accuracy}')
    else:
        # 'val_accuracy'가 없는 경우, 대체 메시지 출력
        print(f'Trial ID: {trial.trial_id} does not have \'val_accuracy\' metric recorded.')


Found 5755 images belonging to 5 classes.


<ipython-input-3-c434948f9a0e>:18: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperParameters as hp


Found 1437 images belonging to 5 classes.
Reloading Tuner from /content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/hpt/ResNet50_hpt_2/tuner0.json

Best Hyperparameters:
{'batch_size': 128, 'units': 640, 'dense_activation': 'tanh', 'dropout_rate': 0.2, 'learning_rate': 0.00694090003577775, 'optimizer': 'sgd'}

Trial ID: 04 does not have 'val_accuracy' metric recorded.
Trial ID: 1 does not have 'val_accuracy' metric recorded.
Trial ID: 0, Best Validation Accuracy: 0.8162475824356079
Trial ID: 05, Best Validation Accuracy: 0.7193208734194437
Trial ID: 3 does not have 'val_accuracy' metric recorded.
Trial ID: 07, Best Validation Accuracy: 0.8953363498051962
Trial ID: 06, Best Validation Accuracy: 0.4294003943602244
Trial ID: 2, Best Validation Accuracy: 0.8149580955505371
Trial ID: 10, Best Validation Accuracy: 0.7491940855979919
Trial ID: 09, Best Validation Accuracy: 0.7790672779083252
Trial ID: 13, Best Validation Accuracy: 0.8736299276351929
Trial ID: 12, Best Validation

In [4]:
# 트라이얼 데이터와 val_accuracy를 추출하고 정렬
trials_data = []
for trial in tuner.oracle.trials.values():
    if 'val_accuracy' in trial.metrics.metrics:
        val_accuracy = trial.metrics.get_best_value('val_accuracy')
        trials_data.append((trial.trial_id, val_accuracy, trial.hyperparameters.values))

# val_accuracy 기준으로 내림차순 정렬
trials_data.sort(key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
print("Trials sorted by Best Validation Accuracy:")
for trial_id, val_accuracy, hyperparameters in trials_data:
    print(f"Trial ID: {trial_id}, Best Validation Accuracy: {val_accuracy}")
    print("Hyperparameters:")
    for param, value in hyperparameters.items():
        print(f"  {param}: {value}")
    print()


Trials sorted by Best Validation Accuracy:
Trial ID: 07, Best Validation Accuracy: 0.8953363498051962
Hyperparameters:
  batch_size: 128
  units: 640
  dense_activation: tanh
  dropout_rate: 0.2
  learning_rate: 0.00694090003577775
  optimizer: sgd

Trial ID: 13, Best Validation Accuracy: 0.8736299276351929
Hyperparameters:
  batch_size: 64
  units: 832
  dense_activation: tanh
  dropout_rate: 0.2
  learning_rate: 0.0031468652789622942
  optimizer: sgd

Trial ID: 11, Best Validation Accuracy: 0.8456909656524658
Hyperparameters:
  batch_size: 96
  units: 928
  dense_activation: relu
  dropout_rate: 0.1
  learning_rate: 0.00019480904540985537
  optimizer: adam

Trial ID: 0, Best Validation Accuracy: 0.8162475824356079
Hyperparameters:
  batch_size: 32
  units: 512
  dense_activation: tanh
  dropout_rate: 0.0
  learning_rate: 0.00024124275097430977
  optimizer: adam

Trial ID: 2, Best Validation Accuracy: 0.8149580955505371
Hyperparameters:
  batch_size: 96
  units: 800
  dense_activation

In [5]:
# 모든 시도의 성능을 저장할 리스트를 초기화
trials_performance = []

for trial in tuner.oracle.trials.values():
    # 각 시도의 'val_accuracy'를 가져옵니다.
    if 'val_accuracy' in trial.metrics.metrics:
        val_accuracy = trial.metrics.get_best_value('val_accuracy')
        trials_performance.append((trial.trial_id, val_accuracy))

# 성능에 따라 시도들을 정렬합니다 (내림차순).
trials_performance.sort(key=lambda x: x[1], reverse=True)

# 두 번째로 높은 성능을 가진 시도의 정보를 출력합니다.
if len(trials_performance) > 1:
    second_best_trial_id, second_best_val_accuracy = trials_performance[1]
    print(f"Second Best Trial ID: {second_best_trial_id}, Validation Accuracy: {second_best_val_accuracy}")
else:
    print("Not enough trials to determine the second best.")


Second Best Trial ID: 13, Validation Accuracy: 0.8736299276351929


In [6]:
# Trial ID: 13, Best Validation Accuracy: 0.8736299276351929
# Hyperparameters:
#   batch_size: 64
#   units: 832
#   dense_activation: tanh
#   dropout_rate: 0.2
#   learning_rate: 0.0031468652789622942
#   optimizer: sgd
best_hps = tuner.get_best_hyperparameters(num_trials=2)[1]
print(best_hps.values,end='\n\n')

{'batch_size': 64, 'units': 832, 'dense_activation': 'tanh', 'dropout_rate': 0.2, 'learning_rate': 0.0031468652789622942, 'optimizer': 'sgd'}



# 최적의 튜닝으로 이어서 학습

In [7]:
# 최적의 튜닝으로 학습 진행

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
import os
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Input
import keras_tuner
from keras_tuner import RandomSearch
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from kerastuner import HyperParameters as hp
import glob
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from PIL import Image
from tensorflow.keras.preprocessing import image


# 데이터셋 경로
dataset_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/data/current_dataset'

# 이미지 크기 및 배치 크기 설정
img_width, img_height = 224, 224  # ResNet50의 기본 이미지 크기

# 모델 생성
def build_model(hp):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=hp.Int('units', min_value=512, max_value=1024, step=32),
              activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid']))(x)
    x = Dropout(rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1))(x)
    predictions = Dense(5, activation='softmax')(x)

    # 학습률 및 옵티마이저 튜닝
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# 배치사이즈 / ImageDataGenerator / train,val 분류
def create_generators(hp):
    batch_size = hp.Int('batch_size', min_value=32, max_value=128, step=32)

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2,
    )

    train_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, validation_generator

# 하이퍼파라미터 및 튜너 설정
hyperparameters = keras_tuner.HyperParameters()

# 데이터 생성기 생성
train_generator, validation_generator = create_generators(hyperparameters)


# 튜너 설정
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=15,  # 시도할 최대 트라이얼 수
    executions_per_trial=3,  # 각 트라이얼마다의 실행 횟수
    # 튜닝 결과 저장 경로
    directory='/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/hpt',
    # 폴더 이름
    project_name='ResNet50_hpt_2',
    hyperparameters=hyperparameters,
    overwrite=False  # 이전 튜닝 세션을 재개하기 위해 False로 설정
)


# 튜너 객체 상태 불러오기
tuner.reload()

# 최적의 하이퍼파라미터 불러오기
best_hps = tuner.get_best_hyperparameters(num_trials=2)[1]

# 모델 구성 및 추가 학습
model = build_model(best_hps)


# 에포크 수와 파일 저장 경로 설정
epochs = 100  # 원하는 에포크 수로 설정 가능


model_save_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/model/best_restnet4'  # 모델 저장 경로 설정


# 조기 종료 콜백 설정
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)
# 체크포인트 콜백 설정
modelname = "ResNet50_{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint_path = os.path.join(model_save_path, modelname)
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

# 모델 학습
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping_callback, checkpoint_callback]
)

# 모델 평가
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

Found 5755 images belonging to 5 classes.
Found 1437 images belonging to 5 classes.
Reloading Tuner from /content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/hpt/ResNet50_hpt_2/tuner0.json
94765736/94765736 [==============================] - 5s 0us/step
Epoch 1/100
180/180 [==============================] - ETA: 0s - loss: 0.6311 - accuracy: 0.7804 

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Epoch 1: val_accuracy improved from -inf to 0.22408, saving model to /content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/model/best_restnet4/ResNet50_01-0.2241.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


180/180 [==============================] - 4486s 25s/step - loss: 0.6311 - accuracy: 0.7804 - val_loss: 1.9144 - val_accuracy: 0.2241
Epoch 2/100
180/180 [==============================] - ETA: 0s - loss: 0.2552 - accuracy: 0.9142
Epoch 2: val_accuracy improved from 0.22408 to 0.23452, saving model to /content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/model/best_restnet4/ResNet50_02-0.2345.hdf5
180/180 [==============================] - 108s 596ms/step - loss: 0.2552 - accuracy: 0.9142 - val_loss: 2.1029 - val_accuracy: 0.2345
Epoch 3/100
180/180 [==============================] - ETA: 0s - loss: 0.1718 - accuracy: 0.9434
Epoch 3: val_accuracy improved from 0.23452 to 0.37300, saving model to /content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/model/best_restnet4/ResNet50_03-0.3730.hdf5
180/180 [==============================] - 106s 585ms/step - loss: 0.1718 - accuracy: 0.9434 - val_loss: 1.8340 - val_accuracy: 0.3730
Epoch 4/100
180/180 [===============

In [12]:
import os
import re

# 모델 저장 경로
model_save_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/HYUN/model/best_restnet4'

# 체크포인트 파일들의 리스트
checkpoint_files = os.listdir(model_save_path)

# 정확도를 저장할 딕셔너리
accuracy_dict = {}

# 각 파일명에서 정확도 추출
for file in checkpoint_files:
    # 파일명에서 정확도 값을 추출하는 정규 표현식
    match = re.search(r"(\d+\.\d+)\.hdf5$", file)
    if match:
        accuracy = float(match.group(1))
        accuracy_dict[file] = accuracy

# 가장 높은 정확도를 가진 모델 찾기
best_model_file = max(accuracy_dict, key=accuracy_dict.get)
best_accuracy = accuracy_dict[best_model_file]

print(f"Best Model: {best_model_file}")
print(f"Accuracy: {best_accuracy}")


Best Model: ResNet50_09-0.8859.hdf5
Accuracy: 0.8859


In [13]:
import os
from tensorflow.keras.models import load_model


# 모델 불러오기
model = load_model(os.path.join(model_save_path, best_model_file))

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터셋 경로
dataset_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/data/current_dataset'

# 이미지 크기 및 배치 크기 설정
img_width, img_height = 224, 224  # ResNet50의 기본 이미지 크기
batch_size = 64

# 이미지 데이터 생성기
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# 훈련 및 검증 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 5755 images belonging to 5 classes.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# TensorFlow 로깅 레벨 설정 (중간 메시지 숨김)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# 테스트 데이터셋 경로와 모델 경로 설정
test_dataset_path = '/content/drive/MyDrive/00_05_4_daejeon_3/2023.12.26 프로젝트/data/test_dataset'
model = load_model(os.path.join(model_save_path, best_model_file))


# 테스트 데이터셋에 대한 ImageDataGenerator 생성
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# 클래스 인덱스 가져오기
class_indices = test_generator.class_indices
class_names = list(class_indices.keys())

# 모든 예측을 수행하고 결과를 저장
correct_predictions = {}
incorrect_predictions = {}
for _ in range(len(test_generator)):
    x, y = next(test_generator)
    image = x[0]  # 이미지 데이터
    true_label = class_names[np.argmax(y[0])]
    prediction = model.predict(x, verbose=0)
    predicted_class = class_names[np.argmax(prediction)]

    # 여기서 image 대신 x[0]을 저장합니다.
    if true_label == predicted_class:
        if true_label not in correct_predictions:
            correct_predictions[true_label] = []
        correct_predictions[true_label].append((image, predicted_class))  # 이미지 데이터 저장
    else:
        if true_label not in incorrect_predictions:
            incorrect_predictions[true_label] = []
        incorrect_predictions[true_label].append((image, predicted_class))  # 이미지 데이터 저장

# 시각화 함수 정의
def visualize_predictions(predictions, title):
    num_images = sum(len(v) for v in predictions.values())
    plt.figure(figsize=(10, num_images // 4 * 4))  # 가로 10인치, 세로는 이미지 수에 비례하게 조정

    i = 1
    for class_name, images in predictions.items():
        for img, pred_class in images:
            plt.subplot(num_images // 4 + 1, 4, i)
            plt.imshow(img)
            if class_name != pred_class:
                plt.title(f"{class_name} as {pred_class}", size=14, color='red')
                plt.gca().add_patch(plt.Rectangle((0, 0), 223, 223, fill=False, edgecolor='red', lw=2))
            else:
                plt.title(f"Correct: {class_name}", size=14)
            plt.axis('off')
            i += 1
    plt.suptitle(title, size=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # 상단 제목에 여백을 줍니다.
    plt.show()

# 맞춘 예측 시각화
visualize_predictions(correct_predictions, "correct_predictions")

# 틀린 예측 시각화
visualize_predictions(incorrect_predictions, "incorrect_predictions")

In [ ]:
from sklearn.metrics import accuracy_score

# 전체 정확도를 위한 리스트
y_true = []
y_pred = []

# 각 카테고리별 정확도를 위한 딕셔너리
category_true = {category: [] for category in class_names}
category_pred = {category: [] for category in class_names}

# 이미지 데이터를 배치 단위로 불러오기
for _ in range(len(test_generator)):
    x, y = next(test_generator)
    true_category = class_names[np.argmax(y[0])]
    y_true.append(true_category)

    # 모델 예측
    prediction = model.predict(x, verbose=0)
    predicted_category = class_names[np.argmax(prediction)]
    y_pred.append(predicted_category)

    # 카테고리별 리스트에 추가
    category_true[true_category].append(true_category)
    category_pred[true_category].append(predicted_category)

# 전체 정확도 계산
total_accuracy = accuracy_score(y_true, y_pred)

# 각 카테고리별 정확도 계산
category_accuracy = {}
for category in class_names:
    category_accuracy[category] = accuracy_score(category_true[category], category_pred[category])

# 결과 출력
print(f"Total Accuracy: {total_accuracy}")
for category, acc in category_accuracy.items():
    print(f"Accuracy for {category}: {acc}")
